# Cassandra Practice Notebook

In [1]:
# install cassandra drivers : conda install -c conda-forge pandas 

Import Packages


In [2]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [3]:
clstr = Cluster()
session = clstr.connect()

# create Keyspace


In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS bd WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':2
    }
""")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


# create a table for testing

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd.Pateint_list ( 
    patient_id INT, 
    f_name TEXT, 
    l_name TEXT, 
    disease TEXT,
    ward_no INT, 
    PRIMARY KEY(ward_no, disease, patient_id)
);
""")

# load Data

In [13]:
df = pd.read_csv('Patient_list.csv')
df

,ward_no,disease,patient_id,f_name,l_name
0,1,'Flu',1001,'John','Doe'
1,2,'COVID-19',1002,'Jane','Smith'
2,1,'Chickenpox',1003,'James','Brown'
3,3,'Malaria',1004,'Lisa','White'
4,2,'Flu',1005,'Robert','Johnson'
5,4,'COVID-19',1006,'Mary','Lee'
6,3,'Chickenpox',1007,'Linda','Green'
7,1,'Malaria',1008,'Michael','Miller'


In [14]:
for index, row in df.iterrows():
    print(f"patient_id = {row.patient_id}, f_name ={row.f_name}, l_name = {row.l_name}, ward_no = {row.ward_no}, disease = {row.disease}")

patient_id = 1001, f_name ='John', l_name = 'Doe', ward_no = 1, disease = 'Flu'
patient_id = 1002, f_name ='Jane', l_name = 'Smith', ward_no = 2, disease = 'COVID-19'
patient_id = 1003, f_name ='James', l_name = 'Brown', ward_no = 1, disease = 'Chickenpox'
patient_id = 1004, f_name ='Lisa', l_name = 'White', ward_no = 3, disease = 'Malaria'
patient_id = 1005, f_name ='Robert', l_name = 'Johnson', ward_no = 2, disease = 'Flu'
patient_id = 1006, f_name ='Mary', l_name = 'Lee', ward_no = 4, disease = 'COVID-19'
patient_id = 1007, f_name ='Linda', l_name = 'Green', ward_no = 3, disease = 'Chickenpox'
patient_id = 1008, f_name ='Michael', l_name = 'Miller', ward_no = 1, disease = 'Malaria'


kepp in mind:
all string values inside csv file must be inside ' '.Be careful don't use " ".
The sequence of all the column names must match while inserting the value into the table.

In [17]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]});
        """
    )
    session.execute(f"""
        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]});
        """
    )


        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES (1, 'Flu', 1001, 'John', 'Doe');
        

        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES (2, 'COVID-19', 1002, 'Jane', 'Smith');
        

        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES (1, 'Chickenpox', 1003, 'James', 'Brown');
        

        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES (3, 'Malaria', 1004, 'Lisa', 'White');
        

        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES (2, 'Flu', 1005, 'Robert', 'Johnson');
        

        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
        VALUES (4, 'COVID-19', 1006, 'Mary', 'Lee');
        

        INSERT INTO bd.Pateint_list (ward_no, disease, patient_id, f_name, l_name)     
 

In [20]:
rows = session.execute("select (ward_no, disease, patient_id, f_name, l_name) from bd.Pateint_list")
for row in rows:
    print(f"ward_no={row[0][0]}, disease={row[0][1]}, patient_id={row[0][2]}, f_name={row[0][3]}, l_name={row[0][4]}")


ward_no=1, disease=Chickenpox, patient_id=1003, f_name=James, l_name=Brown
ward_no=1, disease=Flu, patient_id=1001, f_name=John, l_name=Doe
ward_no=1, disease=Malaria, patient_id=1008, f_name=Michael, l_name=Miller
ward_no=2, disease=COVID-19, patient_id=1002, f_name=Jane, l_name=Smith
ward_no=2, disease=Flu, patient_id=1005, f_name=Robert, l_name=Johnson
ward_no=4, disease=COVID-19, patient_id=1006, f_name=Mary, l_name=Lee
ward_no=3, disease=Chickenpox, patient_id=1007, f_name=Linda, l_name=Green
ward_no=3, disease=Malaria, patient_id=1004, f_name=Lisa, l_name=White


rows = session.execute("SELECT ward_no, disease, patient_id, f_name, l_name FROM bd.Pateint_list")

for row in rows:
    print(f"ward_no={row[0]}, disease={row[1]}, patient_id={row[2]}, f_name={row[3]}, l_name={row[4]}")


In [26]:
rows = session.execute("select (f_name, l_name) from bd.Pateint_list where ward_no=3 AND patient_id = 1004 ALLOW FILTERING")
for row in rows:
    print(f"f_name={row[0][0]}, l_name={row[0][1]}")


f_name=Lisa, l_name=White


In [28]:
rows = session.execute("select (f_name, l_name) from bd.Pateint_list where ward_no=3 AND disease='Malaria' AND patient_id = 1004 ALLOW FILTERING")
for row in rows:
    print(f"f_name={row[0][0]}, l_name={row[0][1]}")


f_name=Lisa, l_name=White


Both of the above queries are similar but one uses partition and other uses ALLOW FILTERING.

Always choose a partition key based on how you access the data to ensure optimal performance and scalability.